# TP Noté

In [35]:
#Nom/prénom : Dang/Vu My Linh

Pour ce TP, vous devez rendre votre jupyter notebook, en précisant votre nom, à envoyer à grappe@lipn.fr avant 16h. 

**Enregistrez bien votre fichier avant de l'envoyer.**

**Commentez bien votre cote :** Décrivez précisément vos variables/contraintes/fonction objectif en commentaire.

In [1]:
# Librairies à importer pour utiliser JuMP avec le solver GLPK
using JuMP
using GLPK

# Définition de constantes pour le statut de résolution du problème
const OPTIMAL = JuMP.MathOptInterface.OPTIMAL
const INFEASIBLE = JuMP.MathOptInterface.INFEASIBLE
const UNBOUNDED = JuMP.MathOptInterface.DUAL_INFEASIBLE;

La firme RoLiX fabrique trois types de produits à base de plastique : bouteilles, sachets, et gants. Chaque produit nécessite une machine de type différent. La firme dispose de ces machines jusqu'à fin 2022.

Le tableau suivant donne pour chaque produit son coût de production unitaire, son prix de vente, ainsi que la quantité de matière première et de travail que nécessite la production d'une unité de chacun de ces trois biens.

Produit | bouteilles | sachets | gants
-----|----|----|----
Coût de production unitaire (euros) | 6 | 4 | 8
Prix de vente unitaire (euros) | 12 | 8 | 15
Matières Premières (plastique en kg) | 4 | 3 | 4
Heures de travail | 3 | 2 | 6

Chaque semaine, le nombre d'heures de travail disponibles est de 150h et la quantité de matières premières (le plastique) de 160kg.


L'objectif est de déterminer le plan de production conduisant au revenu hebdomadaire maximal. 

## Question 1

Donner la formulation PLNE pour résoudre le problème, en : 

* Proposant un choix de variables pour représenter les décisions à prendre dans ce problème. Pour chaque groupe de variables, indiquer ce qu'elles signifient, ainsi que le type, bornes inférieures et supérieures, ensembles d'appartenance des indices.

* Expliquant à la fois la fonction objective et chacune des (familles de) contraintes.

Utiliser cette formulation pour déterminer le plan de production donnant un revenu hebdomadaire maximal (Veiller à avoir un affichage de la solution compréhensible par le PDG de RoLiX qui n'y connait rien en PLNE).

In [2]:
###############################
#   Saisir votre code ici.    #
###############################

# crée un model
m = Model(GLPK.Optimizer)

# crée des variables B, S, G
@variable(m, B >= 0, Int) # B - nombres des bouteilles
@variable(m, S >= 0, Int) # S - nombres des sachets
@variable(m, G >= 0, Int) # G - nombres des gants

# max revenu hebdomadaire = prix de vente - cout de production
# prix de vente = (12*B + 8*S + 15*G)
# cout de production = (6*B + 4*S + 8*G)
@objective(m, Max, (12*B + 8*S + 15*G) - (6*B + 4*S + 8*G)) 

# constraint heures de travail hedbomadaire = 150
@constraint(m, 3*B + 2*S + 6*G <= 150)

#constraint quantité de matiere plastique = 160
@constraint(m, 4*B + 3*S + 4*G <= 160)

# Pour conclusion
optimize!(m)

status = termination_status(m)

if status == INFEASIBLE
    println("Le problème n'est pas réalisable")
elseif status == UNBOUNDED
    println("Le problème est non borné")
elseif status == OPTIMAL
    println("Revenu hebdomadaire maximal = ", JuMP.objective_value(m))
    println("Solution optimale :")
    println("\t Nombre de bouteilles: ", JuMP.value(B))
    println("\t Nombre de sachets: ", JuMP.value(S))
    println("\t Nombre de gants: ", JuMP.value(G))
else
    println("Problème lors de la résolution")
end

Revenu hebdomadaire maximal = 250.0
Solution optimale :
	 Nombre de bouteilles: 30.0
	 Nombre de sachets: 0.0
	 Nombre de gants: 10.0


## Question 2

#### Question 2.1
Implémenter une formulation générique, prenant comme paramètres :
* $n$ : le nombre de types d'objets à fabriquer
* $C$ : un tableau contenant le coût de production unitaire de chaque objet
* $V$ : un tableau contenant le prix de vente unitaire de chaque objet 
* $P$ : un tableau contenant la quantité de matière première nécessaire à la fabrication de chaque objet
* $H$ : un tableau contenant le nombre d'heures de travail nécessaires à la fabrication de chaque objet
* $Q$ : la quantité de plastique disponible par semaine
* $T$ : le nombre d'heures de travail disponible par semaine

Dans chaque tableau (de taille $n$), on supposera que l'entrée d'indice $i$ est la donnée correspondant à l'objet de type $i$.

Donner un affichage clair (en Julia) de la solution : indiquer pour chaque objet quel nombre en est fabriqué.

In [3]:
###############################
#   Saisir votre code ici.    #
###############################

function formulation_generique(n, C, V, P, H, Q, T)

    # crée des longueurs des tables
    lenC = length(C) # longeurs de table des couts
    lenP = length(P) # longeurs de table des quantite de matiere plastique
    lenV = length(V) # longeurs de table des prix de vente
    lenH = length(H) # # longeurs de table des heures de travail nécessaires
    
    # crée un model
    m = Model(GLPK.Optimizer)
    
    # crée des variables
    @variable(m, x[1:n]>=0, Int) #x[1:n] - nombre des types d'objets à fabriquer
    
    #revenue hebdo max = prix de vente - cout de production
    @objective(m, Max, sum(V[i]*x[i] for i in 1:lenV) - sum(C[i]*x[i] for i in 1:lenC))
    
    # constraint for quantite plastique Q de P
    @constraint(m, sum(P[i]*x[i] for i in 1:lenP) <= Q)
    
    # constraint for heures travail T
    @constraint(m, sum(H[i]*x[i] for i in 1:lenH) <= T)
    
    print(m)
    
    # Pour concluire
    optimize!(m)

    status = termination_status(m)

    if status == INFEASIBLE
        println("Le problème n'est pas réalisable")
    elseif status == UNBOUNDED
        println("Le problème est non borné")
    elseif status == OPTIMAL
        println("Revenu hebdomadaire maximal = ", JuMP.objective_value(m))
        println("Solution optimale :")
        for i in 1:n
            println("\t Nombre de produit $i est fabriqué : ", JuMP.value(x[i])) 
        end
    else
        println("Problème lors de la résolution")
    end
end

formulation_generique (generic function with 1 method)

#### Question 2.2
Retrouver les résultats de la question 1 en utilisant votre formulation.

In [4]:
###############################
#   Saisir votre code ici.    #
###############################

# reprise des données dans la question 1
n = 3 # nombre de types d'objet / de produit
C = [6; 4; 8] # tableau des couts
V = [12; 8; 15] # tableau des prix de vent
P = [4; 3; 4] # tableau des quantite de matiere plastiquee
H = [3; 2; 6] # tableau des heures de travail nécessaires
Q = 160 # quantite de matiere plastique dispo chaque semaine
T = 150 # horaires dispo chaque semaine (hebdomadaire)

# utilise ma formulation
formulation_generique(n, C, V, P, H, Q, T)

Revenu hebdomadaire maximal = 250.0
Solution optimale :
	 Nombre de produit 1 est fabriqué : 30.0
	 Nombre de produit 2 est fabriqué : 0.0
	 Nombre de produit 3 est fabriqué : 10.0


#### Question 2.3

Résoudre l'instance correspondant aux données ci-dessous.

In [5]:
#Données pour 2023 :
n = 5
C = [6;7;19;5;4]
V = [11;16;37;10;10]
P = [4;4;9;4;3]
H = [3;6;11;3;4]
Q = 200
T = 140

140

In [6]:
###############################
#   Saisir votre code ici.    #
###############################

formulation_generique(n, C, V, P, H, Q, T)

Revenu hebdomadaire maximal = 233.0
Solution optimale :
	 Nombre de produit 1 est fabriqué : 27.0
	 Nombre de produit 2 est fabriqué : 0.0
	 Nombre de produit 3 est fabriqué : 1.0
	 Nombre de produit 4 est fabriqué : 16.0
	 Nombre de produit 5 est fabriqué : 0.0


## Question 3

En 2023, la firme ne disposera plus de ces machines et devra les louer. Ces machines peuvent être louées à la semaine aux prix suivants (M1 produit les bouteilles, M2 les sachets, et M3 les gants).

Machine | M1 | M2 |M3
-----|----|----|----
Prix de location (semaine) | 200 | 150 | 100

Intégrer la location des machines au plan de production précédent et déterminer le plan de production conduisant au revenu hebdomadaire maximal. 
(Pour produire chacun d'un des biens, il faut bien sûr avoir loué la machine correspondante !)

In [8]:
############################## 
#   Saisir votre code ici.   #
##############################

# ajouter des machines avec le prix de location
M = [200; 150; 100] # 200 pour M1, 150 pour M2, 100 pour M3

# reprise des données dans la question 1 pour tester
n = 3
C = [6; 4; 8]
V = [12; 8; 15]
P = [4; 3; 4]
H = [3; 2; 6]
Q = 160 
T = 150 

function formulation_location(n, C, V, P, H, Q, T, M)

    # crée des longueurs des tables
    lenC = length(C) # longeurs de table des couts
    lenP = length(P) # longeurs de table des quantite de matiere plastique
    lenV = length(V) # longeurs de table des prix de vente
    lenH = length(H) # longeurs de table des heures de travail nécessaires
    lenM = length(M) # longeurs de table des prix de location de machines
    
    # crée un model
    m = Model(GLPK.Optimizer)
    
    # crée des variables
    @variable(m, x[1:n]>=0, Int) #x[1:n] - nombre des types d'objets à fabriquer

    #= create variable binaire 
        0 si je n'utilise pas => on n'a pas besoin de payer le prix de location de machine
        1 si j'utilise le machine => il faut payer
    =#
    @variable(m, y[1:lenM], Bin)
    
    #revenue hebdo max = prix de vente - cout de production - prix de location machine
    @objective(m, Max, sum(V[i]*x[i] for i in 1:lenV) - sum(C[i]*x[i] for i in 1:lenC) - sum(M[j]*y[j] for j in 1:lenM))
    
    # constraint pour quantite plastique Q de P
    @constraint(m, sum(P[i]*x[i] for i in 1:lenP) <= Q)
    
    # constraint pour heures travail T
    @constraint(m, sum(H[i]*x[i] for i in 1:lenH) <= T)
    
    # constraint pour activer la machine M
    # On suppose que le nombre de produit ne peut pas dépasser 2022 (s'il dépasse, on augmente 2022 pour qu'il ne dépasse pas) (comme ne dépasse pas largeur 240 dans TP4)
    for i in 1:lenM
        #= soit x = 0 , soit x <= 2022 (avec condition nombre de produit (x) <= 2022)
            si nombre de produit (x) = 0 => y = 0 => on n'utilise pas de machine => on n'active pas M
            si nombre de produit (x) <= 2022 => y = 1 => on utilise machine => on active M
        =#
        @constraint(m, x[i] - y[i]*2022 <= 0)
    end

    # affichage le model
    print(m)
    
    # Pour conclure
    optimize!(m)

    status = termination_status(m)

    if status == INFEASIBLE
        println("Le problème n'est pas réalisable")
    elseif status == UNBOUNDED
        println("Le problème est non borné")
    elseif status == OPTIMAL
        println("Revenu hebdomadaire maximal = ", JuMP.objective_value(m))
        println("Solution optimale :")
        for i in 1:n
            println("\t Nombre de produit $i est fabriqué : ", JuMP.value(x[i])) 
        end
        println("Affichage le status des machines :")
        for i in 1:3
            if JuMP.value(y[i]) == 1
                println("\t Machine $i est utilisé.")
            else
                println("\t Machine $i n'est PAS utilisé.")
            end
        end
    else
        println("Problème lors de la résolution")
    end
end

# test la formulation
formulation_location(n, C, V, P, H, Q, T, M)

Revenu hebdomadaire maximal = 75.0
Solution optimale :
	 Nombre de produit 1 est fabriqué : 0.0
	 Nombre de produit 2 est fabriqué : 0.0
	 Nombre de produit 3 est fabriqué : 25.0
Affichage le status des machines :
	 Machine 1 n'est PAS utilisé.
	 Machine 2 n'est PAS utilisé.
	 Machine 3 est utilisé.
